# 1

## Implementación de librerias

In [42]:
import numpy as np
from PIL import Image
from Problem import Problem

# 1.1

## Funciones de descretizacion

In [ ]:
import numpy as np
from PIL import Image

def colorSimilar(color, referencia, tolerancia=20):
    return all(abs(int(c1) - int(c2)) <= tolerancia for c1, c2 in zip(color, referencia))


# Función para clasificar una celda según el color predominante
def clasificarCelda(pixeles):
    conteo = {"0": 0, "1": 0, "S": 0, "G": 0}

    BLANCO = (255, 255, 255)
    NEGRO = (0, 0, 0)
    ROJO = (255, 0, 0)
    VERDE = (0, 255, 0)

    for fila in pixeles:
        for r, g, b in fila:
            if colorSimilar((r, g, b), BLANCO):
                conteo["0"] += 1
            elif colorSimilar((r, g, b), NEGRO):
                conteo["1"] += 1
            elif colorSimilar((r, g, b), ROJO):
                conteo["S"] += 1
            elif colorSimilar((r, g, b), VERDE):
                conteo["G"] += 1

    return max(conteo, key=conteo.get)

def discretizar_imagen(ruta_imagen, tamano_celda):
    imagen = Image.open(ruta_imagen).convert("RGB")
    pixeles = np.array(imagen)
    alto, ancho, _ = pixeles.shape

    filas = alto // tamano_celda
    columnas = ancho // tamano_celda

    laberinto = np.zeros((filas, columnas), dtype=str)

    for y in range(filas):
        for x in range(columnas):
            celda_pixeles = pixeles[y * tamano_celda:(y + 1) * tamano_celda, 
                                    x * tamano_celda:(x + 1) * tamano_celda]

            laberinto[y, x] = clasificarCelda(celda_pixeles)

    return laberinto


## Aplicacion

In [44]:
# Imagen de prueba
ruta = "img/Prueba Lab1.bmp"

# Tamaño de pixel definido
tamano_celda = 10  

# Convertir la imagen en una matriz de celdas
matriz_laberinto = discretizar_imagen(ruta, tamano_celda)

# Mostrar la matriz discretizada
print("\nMatriz del laberinto:")
for fila in matriz_laberinto:
    print(" ".join(fila))


Matriz del laberinto:
1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
0 0 0 0 G G G 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
0 0 0 0 G G G 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0
0 0 0 0 G G G 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0

In [45]:
class MazeProblem(Problem):
    def __init__(self, maze_matrix):
        self.maze = maze_matrix
        self.initial_state = self.find_start()
        self.goal_states = self.find_goals()
        super().__init__(self.initial_state)

    def find_start(self):
        for y, row in enumerate(self.maze):
            for x, value in enumerate(row):
                if value == 'S':
                    return (y, x)
        return None

    def find_goals(self):
        goals = []
        for y, row in enumerate(self.maze):
            for x, value in enumerate(row):
                if value == 'G':
                    goals.append((y, x))
        return goals

    def actions(self, state):
        y, x = state
        actions = []

        if y > 0 and self.maze[y - 1, x] != '1':
            actions.append("UP")
        if y < self.maze.shape[0] - 1 and self.maze[y + 1, x] != '1':
            actions.append("DOWN")
        if x > 0 and self.maze[y, x - 1] != '1':
            actions.append("LEFT")
        if x < self.maze.shape[1] - 1 and self.maze[y, x + 1] != '1':
            actions.append("RIGHT")

        return actions

    def result(self, state, action):
        y, x = state
        if action == "UP":
            return (y - 1, x)
        elif action == "DOWN":
            return (y + 1, x)
        elif action == "LEFT":
            return (y, x - 1)
        elif action == "RIGHT":
            return (y, x + 1)
        return state

    def goal_test(self, state):
        return state in self.goal_states

    def step_cost(self, state, action, new_state):
        # Costo uniforme
        return 1 


In [46]:
problema = MazeProblem(matriz_laberinto)

print("Estado inicial:", problema.initial_state)
print("Estados objetivo:", problema.goal_states)
print("Acciones posibles desde el estado inicial:", problema.actions(problema.initial_state))

Estado inicial: (49, 49)
Estados objetivo: [(5, 4), (5, 5), (5, 6), (6, 4), (6, 5), (6, 6), (7, 4), (7, 5), (7, 6)]
Acciones posibles desde el estado inicial: ['UP', 'DOWN', 'LEFT', 'RIGHT']
